<a href="https://colab.research.google.com/github/JsantosAn/Mestrado/blob/main/Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Web Scraping Google Scholar***

> **Usei 10 links (não peguei o link no botão do final da pagina pq a forma que ele mostra o link e uma bagunça e tenho que pensar em um codigo regex pra apagar muita coisa)**

> **Resumo do codigo - useia biblioteca bs4 pra fazer toda a magica, fiz um for pra percorrer a lista com os links em seguida a biblioteca requests faz a conecta com o site e entra em cada perfil disponivel na pagina são 10 por paginas, logo pega os dados da publicação e o nome dos autores e insire em um DataFrame.**






In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
url_autores = ['https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=JUkPAM2m__8J&astart=10',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=ImIAACXR__8J&astart=20',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=C9pGAALj__8J&astart=30',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=69QAAJfs__8J&astart=40',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=tkYLAOnv__8J&astart=50',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=sZmLAJby__8J&astart=60',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=QgsBACb0__8J&astart=70',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=Ul4PAGP2__8J&astart=80',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=4m4tAM73__8J&astart=90',
               'https://scholar.google.com.br/citations?view_op=search_authors&hl=pt-BR&mauthors=label:ontologies&after_author=RGAeAMX4__8J&astart=100']


def buscar_autores(url):
    try:
        resposta = requests.get(url)
        if resposta.status_code == 200:
            return resposta.text
        else:
            print('Erro de requisição')
    except Exception as error:
        print('erro ao requisitar')
        print(error)

def parsing(resposta):
    try:
        soup = BeautifulSoup(resposta, 'html.parser')
        return soup
    except Exception as error:
        print('error ao fazer o parsing html')
        print(error)

def get_autor_link(resposta):
    soup = BeautifulSoup(resposta, "html.parser")
    itens = soup.findAll(class_="gs_ai_name")
    return [
        {"Autor": iten.get_text().strip(),
         "link": 'https://scholar.google.com.br' + iten.find("a").get("href"),
         }
        for iten in itens]

def encontrar_indice(resposta):
    rows = resposta.find(id="gsc_rsb_st").find_all('tr')[1:]
    lst_data=[]
    for row in rows:
        cell = [i.text.rstrip() for i in row.find_all('td')]
        lst_data.append(cell)    
    return lst_data

def inicio(url_autores):
  autores_name=[]
  autores_citacao=[]
  autores_ind_h=[]
  autores_ind_i=[]
  autores_dados =[]
  resposta_busca = buscar_autores(url_autores)
  if resposta_busca:
      links = get_autor_link(resposta_busca)
      for link in links:
          resposta_autor = buscar_autores(link['link'])
          if resposta_autor:
              soup_perfil = parsing(resposta_autor)
          if soup_perfil:
              lista = encontrar_indice(soup_perfil)    
              autores_name =  link['Autor']
              autores_citacao = re.sub(r"(^.*?,)|(,.*?$)", "",str( lista[0:1]))
              autores_citacao = re.sub(r"(\W)", "",str(autores_citacao))
              autores_ind_h   = re.sub(r"(^.*?),|,(.*?$)", "",str(lista[1:2]))
              autores_ind_h = re.sub(r"(\W)", "",str(autores_ind_h))
              autores_ind_i   = re.sub(r"(^.*?),|,(.*?$)", "",str(lista[2:3]))
              autores_ind_i = re.sub(r"(\W)", "",str(autores_ind_i))
              autores_dados.append( ( autores_name,autores_citacao,autores_ind_h,autores_ind_i))
              
  return autores_dados


In [ ]:
autores_dados = pd.DataFrame()
for i in range(len(url_autores)):
  listas=(inicio(url_autores[i]))
  autores_dados = autores_dados.append(listas,)
autores_dados = autores_dados.rename(columns={0:'Autor',1:'Citacao',2:'Indice h', 3:'Indice i10'})
autores_dados = autores_dados.set_index('Autor')


In [21]:
autores_dados.head(60)

,Citacao,Indice h,Indice i10
Autor,,,
Ian Horrocks,65971,97,366
Deborah L. McGuinness,50657,73,202
Christopher J Mungall,42193,77,150
Mark Musen,39918,89,373
Michelle Gwinn Giglio,36684,42,56
Natasha Noy,31859,66,177
Christian Stoeckert,27202,65,138
Helen Parkinson,26078,55,118
Bijan Parsia,24103,57,164


Analizar os links do google school eles não seguem um padrao normal 